In [25]:
import os
import glob
import random
from copy import copy

import cv2
import numpy as np

### Collage

In [7]:
data_dir = "/Users/konishiike/Dataset/LeafAugmentation/"  # directory of data (leaves(img/mask), backgrounds)
save_dir = "/Users/konishiike/DataSet/LeafAugmentation/maple/"  # directory for save data 


def collage (back, leaves, masks):
    
    num = random.randint(50, 100)
    color_list = [[random.randrange(256) for c in range(3)] for i in range(300)]
    leaf_ids = [random.randrange(len(leaves)) for i in range(num)]
    h, w, _ = back.shape

    ann = np.zeros_like(back)
    color_id = 1
    
    # collage leaves (# num) on background
    for l in leaf_ids:
        
        leaf = cv2.imread(leaves[l])
        mask = cv2.imread(masks[l])
        size_y, size_x, _ = leaf.shape 
    
        # random parameter
        rotation = random.randint(0, 359)
        scale = random.uniform(0.6, 1.2)
        x = random.randrange(-size_x//2, w-size_x//2)
        y = random.randrange(-size_y//2, h-size_y//2)
    
        # transofrmation
        rot = cv2.getRotationMatrix2D((size//2, size//2), rotation, scale)
        pos = np.float32([[1, 0, x],
                          [0, 1, y]])
        leaf = cv2.warpAffine(leaf, rot, (w, h), flags=cv2.INTER_NEAREST)
        leaf = cv2.warpAffine(leaf, pos, (w, h), flags=cv2.INTER_NEAREST)
        mask = cv2.warpAffine(mask, rot, (w, h), flags=cv2.INTER_NEAREST)
        mask = cv2.warpAffine(mask, pos, (w, h), flags=cv2.INTER_NEAREST)
        
        # collage
        np.putmask(back, mask, leaf)
        np.putmask(ann, mask, color_list[color_id])
        color_id += 1
        
    return back, ann


# backgrpund images
back_list = glob.glob(os.path.join(data_dir, "backgrounds/*.png"))
back_ids = [random.randrange(len(back_list)) for i in range(number)]

img_num = 1
for b in back_ids:
    
    leaf_list = glob.glob(os.path.join(data_dir, "leaves", "img/*.png"))
    leaf_list.sort()
    mask_list = glob.glob(os.path.join(data_dir, "leaves", "mask/*.png"))
    mask_list.sort()
    
    back = cv2.imread(backgrounds_list[b])
    img, ann= collage(back, list1, mask_list1)
    
    os.makedirs(os.path.join(save_dir, "img"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "annotation"), exist_ok=True)
    cv2.imwrite(os.path.join(save_dir, "img", "{:04}.png".format(img_num)), img)
    cv2.imwrite(os.path.join(save_dir, "annotation", "{:04}.png".format(img_num)), ann)
    img_num += 1